In [23]:
!nvidia-smi

Wed Sep 25 13:14:37 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060 ...    Off |   00000000:01:00.0  On |                  N/A |
| N/A   49C    P0             35W /  125W |    1530MiB /   8188MiB |     42%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [24]:
import torch
from torch import nn
torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

In [25]:
torch.cuda.device_count()

1

In [26]:
def try_gpu(i=0):  #@save
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():  #@save
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

In [27]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [28]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [29]:
Y = torch.rand(2, 3, device=try_gpu(0))
Y

tensor([[0.9389, 0.7050, 0.0846],
        [0.8281, 0.3003, 0.4224]], device='cuda:0')

In [30]:
Z = X.cuda(0)
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')


In [31]:
Y + Z

tensor([[1.9389, 1.7050, 1.0846],
        [1.8281, 1.3003, 1.4224]], device='cuda:0')

In [32]:
Z.cuda(0) is Z

True

In [33]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

In [34]:
net(X)

tensor([[0.8259],
        [0.8259]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [35]:
net[0].weight.data.device

device(type='cuda', index=0)